In [3]:
# %%
from pathlib import Path
import os

import pandas as pd
import numpy as np

import torch
from torch import nn

# from data_frame import *
# from data import *

In [4]:
data_cols = ['x', 'x_len', 'c', 'c_len', 'w_id']
data_dir = "processed"
data = []

for col in data_cols:
    data.append(
        torch.from_numpy(
            np.load(os.path.join(data_dir, '{}.npy'.format(col)))
        )
    )
print(data[0].shape)

torch.Size([11615, 1200, 3])


In [5]:
data[0][0, :, :]

tensor([[ 0.0000,  0.0000,  1.0000],
        [ 0.1970, -0.2660,  0.0000],
        [ 0.1404, -0.1231,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]])

In [8]:
ALPHABET = [
    '\x00', ' ', '!', '"', '#', "'", '(', ')', ',', '-', '.',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';',
    '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K',
    'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y',
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
    'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
    'y', 'z'
]
len(ALPHABET)

73

In [27]:
m = nn.Sigmoid()
input = torch.randn(2)
output = m(input)

In [10]:
DR = WriterDataReader("processed")

train size 56
val size 3
test size 59


In [26]:
gen = DR.train_batch_generator(2)
for d in gen:
    print(d['x'])
    break

tensor([[[ 0.0000,  0.0000,  1.0000],
         [-0.0096, -0.1607,  0.0000],
         [-0.0336, -0.3333,  0.0000],
         ...,
         [ 0.1535, -0.1487,  0.0000],
         [ 0.0959,  0.2974,  0.0000],
         [ 0.0168,  0.4556,  1.0000]],

        [[ 0.0000,  0.0000,  1.0000],
         [-0.1593,  0.0774,  0.0000],
         [-0.2456,  0.0752,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]]], device='cuda:0')


In [25]:
D = DataReader("processed")
for d in D.train_batch_generator(2):
    print(d['x'])
    break

train size 11034
val size 581
test size 11615
tensor([[[ 0.0000,  0.0000,  1.0000],
         [-0.0902, -0.0391,  0.0000],
         [-0.0090, -0.0661,  0.0000],
         ...,
         [ 1.0788,  0.6641,  0.0000],
         [ 0.8865,  0.3937,  0.0000],
         [ 0.5770,  0.1533,  1.0000]],

        [[ 0.0000,  0.0000,  1.0000],
         [-0.1063,  0.0792,  0.0000],
         [-0.0667,  0.1001,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]]], device='cuda:0')


In [9]:
from __future__ import print_function
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d


alphabet = [
    '\x00', ' ', '!', '"', '#', "'", '(', ')', ',', '-', '.',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';',
    '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K',
    'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y',
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
    'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
    'y', 'z'
]
alphabet_ord = list(map(ord, alphabet))
alpha_to_num = defaultdict(int, list(map(reversed, enumerate(alphabet))))
num_to_alpha = dict(enumerate(alphabet_ord))

MAX_STROKE_LEN = 1200
MAX_CHAR_LEN = 75


def align(coords):
    """
    corrects for global slant/offset in handwriting strokes
    """
    coords = np.copy(coords)
    X, Y = coords[:, 0].reshape(-1, 1), coords[:, 1].reshape(-1, 1)
    X = np.concatenate([np.ones([X.shape[0], 1]), X], axis=1)
    offset, slope = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(Y).squeeze()
    theta = np.arctan(slope)
    rotation_matrix = np.array(
        [[np.cos(theta), -np.sin(theta)],
         [np.sin(theta), np.cos(theta)]]
    )
    coords[:, :2] = np.dot(coords[:, :2], rotation_matrix) - offset
    return coords


def skew(coords, degrees):
    """
    skews strokes by given degrees
    """
    coords = np.copy(coords)
    theta = degrees * np.pi/180
    A = np.array([[np.cos(-theta), 0], [np.sin(-theta), 1]])
    coords[:, :2] = np.dot(coords[:, :2], A)
    return coords


def stretch(coords, x_factor, y_factor):
    """
    stretches strokes along x and y axis
    """
    coords = np.copy(coords)
    coords[:, :2] *= np.array([x_factor, y_factor])
    return coords


def add_noise(coords, scale):
    """
    adds gaussian noise to strokes
    """
    coords = np.copy(coords)
    coords[1:, :2] += np.random.normal(loc=0.0, scale=scale, size=coords[1:, :2].shape)
    return coords


def encode_ascii(ascii_string):
    """
    encodes ascii string to array of ints
    """
    return np.array(list(map(lambda x: alpha_to_num[x], ascii_string)) + [0])


def denoise(coords):
    """
    smoothing filter to mitigate some artifacts of the data collection
    """
    coords = np.split(coords, np.where(coords[:, 2] == 1)[0] + 1, axis=0)
    new_coords = []
    for stroke in coords:
        if len(stroke) != 0:
            x_new = savgol_filter(stroke[:, 0], 7, 3, mode='nearest')
            y_new = savgol_filter(stroke[:, 1], 7, 3, mode='nearest')
            xy_coords = np.hstack([x_new.reshape(-1, 1), y_new.reshape(-1, 1)])
            stroke = np.concatenate([xy_coords, stroke[:, 2].reshape(-1, 1)], axis=1)
            new_coords.append(stroke)

    coords = np.vstack(new_coords)
    return coords


def interpolate(coords, factor=2):
    """
    interpolates strokes using cubic spline
    """
    coords = np.split(coords, np.where(coords[:, 2] == 1)[0] + 1, axis=0)
    new_coords = []
    for stroke in coords:

        if len(stroke) == 0:
            continue

        xy_coords = stroke[:, :2]

        if len(stroke) > 3:
            f_x = interp1d(np.arange(len(stroke)), stroke[:, 0], kind='cubic')
            f_y = interp1d(np.arange(len(stroke)), stroke[:, 1], kind='cubic')

            xx = np.linspace(0, len(stroke) - 1, factor*(len(stroke)))
            yy = np.linspace(0, len(stroke) - 1, factor*(len(stroke)))

            x_new = f_x(xx)
            y_new = f_y(yy)

            xy_coords = np.hstack([x_new.reshape(-1, 1), y_new.reshape(-1, 1)])

        stroke_eos = np.zeros([len(xy_coords), 1])
        stroke_eos[-1] = 1.0
        stroke = np.concatenate([xy_coords, stroke_eos], axis=1)
        new_coords.append(stroke)

    coords = np.vstack(new_coords)
    return coords


def normalize(offsets):
    """
    normalizes strokes to median unit norm
    """
    offsets = np.copy(offsets)
    offsets[:, :2] /= np.median(np.linalg.norm(offsets[:, :2], axis=1))
    return offsets


def coords_to_offsets(coords):
    """
    convert from coordinates to offsets
    """
    offsets = np.concatenate([coords[1:, :2] - coords[:-1, :2], coords[1:, 2:3]], axis=1)
    offsets = np.concatenate([np.array([[0, 0, 1]]), offsets], axis=0)
    return offsets


def offsets_to_coords(offsets):
    """
    convert from offsets to coordinates
    """
    return np.concatenate([np.cumsum(offsets[:, :2], axis=0), offsets[:, 2:3]], axis=1)


def draw(
        offsets,
        ascii_seq=None,
        align_strokes=True,
        denoise_strokes=True,
        interpolation_factor=None,
        save_file=None
):
    strokes = offsets_to_coords(offsets)

    if denoise_strokes:
        strokes = denoise(strokes)

    if interpolation_factor is not None:
        strokes = interpolate(strokes, factor=interpolation_factor)

    if align_strokes:
        strokes[:, :2] = align(strokes[:, :2])

    fig, ax = plt.subplots(figsize=(12, 3))

    stroke = []
    for x, y, eos in strokes:
        stroke.append((x, y))
        if eos == 1:
            coords = zip(*stroke)
            ax.plot(coords[0], coords[1], 'k')
            stroke = []
    if stroke:
        coords = zip(*stroke)
        ax.plot(coords[0], coords[1], 'k')
        stroke = []

    ax.set_xlim(-50, 600)
    ax.set_ylim(-40, 40)

    ax.set_aspect('equal')
    plt.tick_params(
        axis='both',
        left='off',
        top='off',
        right='off',
        bottom='off',
        labelleft='off',
        labeltop='off',
        labelright='off',
        labelbottom='off'
    )

    if ascii_seq is not None:
        if not isinstance(ascii_seq, str):
            ascii_seq = ''.join(list(map(chr, ascii_seq)))
        plt.title(ascii_seq)

    if save_file is not None:
        plt.savefig(save_file)
        print('saved to {}'.format(save_file))
    else:
        plt.show()
    plt.close('all')


In [13]:
import svgwrite


def _draw(strokes, lines, filename, stroke_colors=None, stroke_widths=None):
    stroke_colors = stroke_colors or ['black']*len(lines)
    stroke_widths = stroke_widths or [2]*len(lines)

    line_height = 60
    view_width = 1000
    view_height = line_height*(len(strokes) + 1)

    dwg = svgwrite.Drawing(filename=filename)
    dwg.viewbox(width=view_width, height=view_height)
    dwg.add(dwg.rect(insert=(0, 0), size=(view_width, view_height), fill='white'))

    initial_coord = np.array([0, -(3*line_height / 4)])
    for offsets, line, color, width in zip(strokes, lines, stroke_colors, stroke_widths):

        if not line:
            initial_coord[1] -= line_height
            continue

        offsets[:, :2] *= 1.5
        strokes = offsets_to_coords(offsets)
        strokes = denoise(strokes)
        strokes[:, :2] = align(strokes[:, :2])

        strokes[:, 1] *= -1
        strokes[:, :2] -= strokes[:, :2].min() + initial_coord
        strokes[:, 0] += (view_width - strokes[:, 0].max()) / 2

        prev_eos = 1.0
        p = "M{},{} ".format(0, 0)
        for x, y, eos in zip(*strokes.T):
            p += '{}{},{} '.format('M' if prev_eos == 1.0 else 'L', x, y)
            prev_eos = eos
        path = svgwrite.path.Path(p)
        path = path.stroke(color=color, width=width, linecap='round').fill("none")
        dwg.add(path)

        initial_coord[1] -= line_height

    dwg.save()

In [7]:
sample_idx = 0
x = data[0][sample_idx]
x_len = data[1][sample_idx]
c = data[2][sample_idx]
c_len = data[3][sample_idx]

tensor([[ 0.0000,  0.0000,  1.0000],
        [ 0.1970, -0.2660,  0.0000],
        [ 0.1404, -0.1231,  0.0000],
        ...,
        [-0.1428,  1.1131,  0.0000],
        [-0.3103,  0.8447,  1.0000],
        [ 0.0000,  0.0000,  0.0000]])

In [14]:
sample_idx = 0
x = data[0][sample_idx]
x_len = data[1][sample_idx]
c = data[2][sample_idx]
c_len = data[3][sample_idx]
print(x_len, c_len)
print(x.shape, [ALPHABET[i] for i in c[:c_len]])
print(    "".join([ALPHABET[i] for i in c[:c_len]]))

x = x[:x_len]
print(x.shape)
print(x_len)

lines = [
    "".join([ALPHABET[i] for i in c[:c_len]])
]
biases = [.75 for i in lines]
styles = [9 for i in lines]
strokes = x.unsqueeze(0)
filename = "out.svg"

_draw(strokes, lines, filename, stroke_colors=None, stroke_widths=None)



tensor(612, dtype=torch.int16) tensor(36, dtype=torch.int8)
torch.Size([1200, 3]) ['"', 'I', ' ', "'", 'l', 'l', ' ', 't', 'r', 'y', ' ', 't', 'o', ' ', 'r', 'e', 'm', 'e', 'm', 'b', 'e', 'r', '.', ' ', 'S', 'h', 'a', 'l', 'l', ' ', 'w', 'e', ' ', 'g', 'o', '\x00']
"I 'll try to remember. Shall we go 
torch.Size([612, 3])
tensor(612, dtype=torch.int16)


In [94]:
# %%
dr = DataReader("processed")
# %%
dr.train_df.dtypes()
# torch.tensor(dr.train_df['c'])
# %%

train size 11034
val size 581
test size 11615


x        float32
x_len      int16
c           int8
c_len       int8
dtype: object

In [3]:
def get_tgt_mask(size) -> torch.tensor:
    # Generates a squeare matrix where the each row allows one word more to be seen
    mask = torch.tril(torch.ones(size, size) == 1) # Lower triangular matrix
    mask = mask.float()
    mask = mask.masked_fill(mask == 0, float('-inf')) # Convert zeros to -inf
    mask = mask.masked_fill(mask == 1, float(0.0)) # Convert ones to 0

    # EX for size=5:
    # [[0., -inf, -inf, -inf, -inf],
    #  [0.,   0., -inf, -inf, -inf],
    #  [0.,   0.,   0., -inf, -inf],
    #  [0.,   0.,   0.,   0., -inf],
    #  [0.,   0.,   0.,   0.,   0.]]

    return mask

def create_pad_mask(matrix: torch.tensor, pad_token: int) -> torch.tensor:
    # If matrix = [1,2,3,0,0,0] where pad_token=0, the result mask is
    # [False, False, False, True, True, True]
    return (matrix == pad_token)

In [17]:
batch_x = None
batch_c = None
for i in dr.train_batch_generator(11):
    batch = i
    print(i['x'].shape)
    print(i['c'].shape)
    batch_x = i['x']
    batch_c = i['c']
    # print(i['c'])
    print()
    break



(11, 950, 3)
(11, 42)



In [15]:
create_pad_mask(batch_c, torch.tensor([0, 0, 0]))

False

In [58]:
pad_mask = torch.ones_like(torch.tensor(batch['x']))
for i, l in enumerate(batch['x_len']):
    print(i, l)
    pad_mask[i, l:] = 0

0 594
1 849
2 513
3 569
4 632
5 505
6 950
7 419
8 556
9 702
10 678


In [59]:
(torch.tensor(batch['x']) * pad_mask == torch.tensor(batch['x'])).all()

tensor(True)

In [70]:
# an Embedding module containing 10 tensors of size 3
embedding = nn.Embedding(10, 3)
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([[1,2,4,5],[4,3,2,9]])
print(embedding(input))
print(embedding(input).shape)


# # example with padding_idx
# embedding = nn.Embedding(10, 3, padding_idx=0)
# input = torch.LongTensor([[0,2,0,5]])
# embedding(input)

# # example of changing `pad` vector
# padding_idx = 0
# embedding = nn.Embedding(3, 3, padding_idx=padding_idx)
# embedding.weight
# with torch.no_grad():
#     embedding.weight[padding_idx] = torch.ones(3)
# embedding.weight

tensor([[[ 0.6020, -0.0879,  0.4007],
         [ 0.8236, -1.1303, -0.4993],
         [-0.3771, -0.8300, -0.2342],
         [ 0.1727, -1.9924, -0.3265]],

        [[-0.3771, -0.8300, -0.2342],
         [ 1.5637,  0.5562,  0.6208],
         [ 0.8236, -1.1303, -0.4993],
         [ 0.9442, -0.6144, -0.9216]]], grad_fn=<EmbeddingBackward0>)
torch.Size([2, 4, 3])


In [80]:
from torch.nn import functional as F
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
loss = F.binary_cross_entropy_with_logits(input, target)
loss.backward()

In [83]:
target

tensor([0., 0., 0.])

In [84]:
input

tensor([-0.4580, -1.0515,  0.7332], requires_grad=True)

In [91]:
        data_cols = ['x', 'x_len', 'c', 'c_len']
        data = [torch.tensor(np.load(os.path.join(data_dir, '{}.npy'.format(i)))) for i in data_cols]

In [93]:
data[2]

tensor([[ 3, 32,  1,  ...,  0,  0,  0],
        [61, 69, 23,  ...,  0,  0,  0],
        [41, 54, 51,  ...,  0,  0,  0],
        ...,
        [49, 54, 47,  ...,  0,  0,  0],
        [53, 47, 64,  ...,  0,  0,  0],
        [49, 61, 59,  ...,  0,  0,  0]], dtype=torch.int8)